In [1]:
#import various useful libraries
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
import cellpose
from cellpose import core, utils, io, models, metrics
from glob import glob

from skimage import io as skio

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [2]:
# check cuda version & that the gpu is working in this environment
!nvcc --version
!nvidia-smi

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Tue_Feb_27_16:28:36_Pacific_Standard_Time_2024
Cuda compilation tools, release 12.4, V12.4.99
Build cuda_12.4.r12.4/compiler.33961263_0
Tue Jul  2 10:16:39 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.86                 Driver Version: 551.86         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti   WDDM  |   00000000:03:00.0  On |                  N/A |
|  0% 

In [2]:
# run this
import os
from cellpose import denoise, io
import glob
import skimage.io as skio
import numpy as np
import mrc
# Set up logging for cellpose
io.logger_setup()

# Create the denoising model
modeldeldia = denoise.DenoiseModel(gpu=True, model_type="denoise_cyto3", diam_mean=30)

# Function to denoise files in a folder using the provided model
def denoise_folder(folder, model):
    # Create the 'denoised' subfolder if it doesn't exist
    denoised_folder = os.path.join(folder, "denoised")
    os.makedirs(denoised_folder, exist_ok=True)

    # Get a list of all files in the folder
    all_files = glob.glob(os.path.join(folder, '*'))

    # Iterate through each file in the folder
    for current in all_files:
        print('Now processing:', current)

        # Read video or image
        if current.endswith('.dv'):
            # Read DV 
            image = mrc.imread(current)
        elif current.endswith('.tif'):
            # Read TIFF image using skimage
            image = skio.imread(current)
        else:
            print("Not doing this one:", current)
            continue
        # Perform denoising
        denoised = model.eval(image, batch_size=24, channels=[0,0], z_axis=0)

        # Remove trailing axis
        denoised_squeezed = np.squeeze(denoised, axis=-1)

        # Save the denoised image in the 'denoised' subfolder
        file_name = os.path.basename(current)
        base_name = os.path.splitext(file_name)[0]
        tiff_filename = os.path.join(denoised_folder, base_name + "_denoised.tif")
        skio.imsave(tiff_filename, denoised_squeezed, check_contrast=False)
        print('Saved:', tiff_filename)

# Folder containing files to denoise
folder = 'D:/omicron/vids/'

# Call the denoise_folder function
denoise_folder(folder, modeldeldia)

ImportError: DLL load failed while importing _imaging: The specified module could not be found.